In [1]:
import pandas as pd

In [13]:
df = pd.DataFrame({
    'admin0': ['cntry1']*6+['cntry2']*5,
    'admin1': ['state1']*3+['state2']*3+['state3']*4+['state4'],
    'admin2': ['city1']*2+['city2']+['city3']+['city4']*2+['city5']+['city6']*3+['city6'],
    'windspeed': ['60km/h', '90km/h', '60km/h', '60km/h', '60km/h', '90km/h', '60km/h', '60km/h', '90km/h', '120km/h', '60km/h'],
    'population': [700, 210, 100, 70, 180, 370, 890, 120, 420, 360, 740]
})

In [14]:
df

,admin0,admin1,admin2,population,windspeed
0,cntry1,state1,city1,700,60km/h
1,cntry1,state1,city1,210,90km/h
2,cntry1,state1,city2,100,60km/h
3,cntry1,state2,city3,70,60km/h
4,cntry1,state2,city4,180,60km/h
5,cntry1,state2,city4,370,90km/h
6,cntry2,state3,city5,890,60km/h
7,cntry2,state3,city6,120,60km/h
8,cntry2,state3,city6,420,90km/h
9,cntry2,state3,city6,360,120km/h
